# Adding Golf Round
This notebook will be used to build the functions needed to add a golf course round to the database

In [1]:
import psycopg2
from psycopg2 import Error
import pandas as pd
import streamlit as st

# Variables
USER = "postgres"
PASSWORD = "Barca2011"
DATABASE = "golf_dashboard_db"

### All Functions (so far)

In [6]:
# Initialise connection and generate cursor
def connect_to_postgres_database(user, password, database, host="127.0.0.1", port="5432"):
    """
    Function connects to a database and returns the cursor object
    :param user: database username
    :param password: database password
    :param database: database name
    :param host: server location
    :param port: listening port
    :return: psycopg2 cursor object
    """
    try:
        con = psycopg2.connect(user=user,
                               password=password,
                               database=database,
                               host=host,
                               port=port)
        cursor = con.cursor()
    except (Exception, Error) as error:
        print("Error while connecting to PostgreSQL", error)
    return con, cursor


# View queried table as dataframe
def table_to_dataframe(table):
    """
    Function returns a queried table in a Pandas DataFrame
    :param table: name of table in database
    :return: dataframe of table
    """
    cursor.execute("""SELECT * FROM {}""".format(table))
    tmp = cursor.fetchall()
    col_names = list()
    for elt in cursor.description:
        col_names.append(elt[0])
    df = pd.DataFrame(tmp, columns=col_names)
    return df


# Cursor execute command
def cursor_execute_tuple(command, data_tuple):
    """
    Function uses the cursor object to execute a command with a tuple pair. It commits and rollsback if error
    :param command: SQL query to be executed
    :param data_tuple: data pairing for SQL query variables
    :return:
    """
    try:
        cursor.execute(command, data_tuple)
        con.commit()
        print("Successfully executed the command")
    except:
        con.rollback()
        print("Could not successfully execute the command")
    return None


# Insert course data into course table
def insert_course_in_course_table(name, holes_18, city, slope, rating, par, country):
    """
    Function inserts course information into the course table
    :param name: the name of the course (TEXT)
    :param holes_18: the number of holes the course has (INT)
    :param city: the city/location of the course (TEXT)
    :param slope: the slope of the course (FLOAT)
    :param rating: the rating of the course (FLOAT)
    :param par: the par of the course (INT)
    :param country: the country of the course (TEXT)
    :return:
    """
    insert_command = """INSERT INTO course
                  (name, holes_18, city, slope, rating,  par, country)
                  VALUES (%s, %s, %s, %s, %s, %s, %s);"""
    data_tuple = (name, holes_18, city, slope, rating, par, country)
    cursor_execute_tuple(insert_command, data_tuple)
    return None


def get_id_from_course_name(course_name):
    """
    Function returns the id of the course in the course table based on the name
    :param course_name: name of course quiered
    :return: id of course quiered
    """
    insert_command = """SELECT id FROM course
                    WHERE name = %s;"""
    cursor.execute(insert_command, [course_name])
    returned_value = cursor.fetchall()
    id = returned_value[0][0]
    return id


# Score card generate and download
def make_hole_number_range_scorecard(course_holes18):
    """
    Function creates a list of numbers. Either 9 holes or 18
    :param course_holes18: number of holes the course has
    :return: list of holes where number of holes is either 9 or 18
    """
    if course_holes18 == 9:
        list_of_holes = list(range(1, 10))
    elif course_holes18 == 18:
        list_of_holes = list(range(1, 19))
    else:
        pass
    return list_of_holes


def make_course_score_card_csv(course_name, course_holes18):
    """
    Function will generate a .csv containing the course name and number of holes
    :param course_name: name of course quiered
    :param course_holes18: number of holes the course has
    :return: to_csv object that can be downloaded
    """
    course_score_card_template_df = pd.DataFrame()
    list_of_holes = make_hole_number_range_scorecard(course_holes18)
    course_score_card_template_df["Hole"] = list_of_holes
    course_score_card_template_df["Distance"] = ""
    course_score_card_template_df["Par"] = ""
    course_score_card_template_df["Stroke Index"] = ""
    course_score_card_template_df["Name"] = course_name
    course_score_card_template_df_csv = course_score_card_template_df.to_csv(index=False)
    return course_score_card_template_df_csv


# Insert course features par/distance/si
def make_data_tuple_9holes(table, course_feature, course_id):
    """
    Function creates the SQL command needed to insert features into a table - 9 holes
    :param table: name of table in database
    :param course_feature: list of features
    :param course_id: id of course in course table
    :return: insert SQL command, features as tuple
    """
    insert_command = """INSERT INTO {}
                  (course_id, hole1, hole2, hole3, hole4, hole5, hole6, hole7, hole8, hole9)
                  VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s);""".format(table)
    course_id_list = [course_id]
    course_id_tuple = tuple(course_id_list)
    feature_tuple = tuple(course_feature)
    data_tuple = tuple(course_id_tuple) + tuple(feature_tuple)
    return insert_command, data_tuple


def make_data_tuple_18holes(table, course_feature, course_id):
    """
    Function creates the SQL command needed to insert features into a table - 18 holes
    :param table: name of table in database
    :param course_feature: list of features
    :param course_id: id of course in course table
    :return: insert SQL command, features as tuple
    """
    insert_command = """INSERT INTO {}
                      (course_id, hole1, hole2, hole3, hole4, hole5, hole6, hole7, hole8, hole9, hole10, hole11, hole12, hole13, hole14, hole15, hole16, hole17, hole18)
                      VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s);""".format(
        table)
    course_id_list = [course_id]
    course_id_tuple = tuple(course_id_list)
    feature_tuple = tuple(course_feature)
    data_tuple = tuple(course_id_tuple) + tuple(feature_tuple)
    return insert_command, data_tuple


def insert_score_card_feature_to_table(table, course_feature, course_id):
    """
    Function pipelines the process to insert 9/18 hole feature information (par, si, distance) into a database table
    :param table: name of table in database
    :param course_feature: list of features
    :param course_id: id of course in course table
    :return:
    """
    if len(course_feature) == 9:
        insert_command, data_tuple = make_data_tuple_9holes(table, course_feature, course_id)
    elif len(course_feature) == 18:
        insert_command, data_tuple = make_data_tuple_18holes(table, course_feature, course_id)
    cursor_execute_tuple(insert_command, data_tuple)
    return None

### Establish Connection

In [14]:
con, cursor = connect_to_postgres_database(USER, PASSWORD, DATABASE, host="127.0.0.1",
                                           port="5432")

### Return Course Features via Course Name

In [18]:
course_name = "Red Mountain Golf Club"
get_id_from_course_name(course_name)

5

In [96]:
def make_course_feature_using_course_id(course_id, table, holes="18"):
    """
    Function returns the row for a course feature table based on the id passed. It returns the specified number of holes 18, Front 9 or Back 9 based on the "holes" parameter
    :param course_id: id of course in course table
    :return: list of course features
    """
    insert_command = """SELECT * FROM {}
                WHERE course_id = %s;""".format(table)
    cursor.execute(insert_command, [course_id])
    returned_value = cursor.fetchall()
    feature_list = list(returned_value[0])
    if holes == "Front 9":
        feature_list = feature_list[:9]
    elif holes == "Back 9":
        feature_list = feature_list[10:]
    else:
        pass
    return feature_list

In [98]:
course_distance_list = make_course_feature_using_course_id(80, "course_distance", "Front 9")
course_par_list = make_course_feature_using_course_id(80, "course_par")
course_si_list = make_course_feature_using_course_id(80, "course_stroke_index")
course_distance_list

[80, 25, 75, 125, 175, 225, 275, 325, 375]

In [35]:
DISTANCE_TABLE = "course_distance"
PAR_TABLE = "course_par"
SI_TABLE = "course_stroke_index"

def make_all_course_feature_lists(course_id):
    """
    Function pipelines all three calls to the three course feature tables
    :param course_id: id of course in course table
    :return: three lists made of distance, par, si
    """
    course_distance_list = make_course_feature_using_course_id(course_id, DISTANCE_TABLE)
    course_par_list = make_course_feature_using_course_id(course_id, PAR_TABLE)
    course_si_list = make_course_feature_using_course_id(course_id, SI_TABLE)
    return course_distance_list, course_par_list, course_si_list

In [39]:
course_distance_list, course_par_list, course_si_list = make_all_course_feature_lists(80)

In [40]:
temp_df = pd.DataFrame()
temp_df["Distance"] = course_distance_list
temp_df["HCP"] = course_si_list
temp_df["Par"] = course_par_list

In [92]:
temp_df

,Distance,HCP,Par
0,80,80,80
1,25,456,2
2,75,654,4
3,125,852,6
4,175,1050,8
5,225,1248,10
6,275,1446,12
7,325,1644,14
8,375,1842,16
9,425,2040,18


In [ ]:
def make_round_score_card_csv(course_distance_list, course_si_list, course_par_list):
    """
    Function will generate a .csv containing the course distance/si/par along with blank entries for strokes/putts/fir/gir
    :param course_distance_list: the distances of each hole on the course
    :param course_si_list: the stroke index of each hole on the course
    :param course_par_list: the par of each hole on the course
    :return: to_csv object that can be downloaded
    """
    round_score_card_template_df = pd.DataFrame()
    round_score_card_template_df["Distance"] = course_distance_list
    round_score_card_template_df["Stroke Index"] = course_si_list
    round_score_card_template_df["Par"] = course_par_list
    round_score_card_template_df["Shots"] = ""
    round_score_card_template_df["Putts"] = ""
    round_score_card_template_df["FIR"] = ""
    round_score_card_template_df["GIR"] = ""
    round_score_card_template_df_csv = round_score_card_template_df.to_csv(index=False)
    return round_score_card_template_df_csv

In [111]:
DISTANCE_TABLE = "course_distance"
PAR_TABLE = "course_par"
SI_TABLE = "course_stroke_index"


def make_course_feature_using_course_id(course_id, table, holes="18"):
    """
    Function returns the row for a course feature table based on the id passed. It returns the specified number of holes 18, Front 9 or Back 9 based on the "holes" parameter
    :param course_id: id of course in course table
    :return: list of course features
    """
    insert_command = """SELECT * FROM {}
                WHERE course_id = %s;""".format(table)
    cursor.execute(insert_command, [course_id])
    returned_value = cursor.fetchall()
    feature_list = list(returned_value[0])
    if holes == "Front 9":
        feature_list = feature_list[1:10]
    elif holes == "Back 9":
        feature_list = feature_list[10:]
    else:
        feature_list = feature_list[1:]
    return feature_list

def make_hole_number_list(holes):
    """
    Function creates a list of numbers. Either 1-9, 10-18 holes or 1-18
    :param holes: number of holes the course has
    :return: list of holes where number of holes is either 1-9, 10-18 holes or 1-18
    """
    if holes == "Front 9":
        list_of_holes = list(range(1, 10))
    elif holes == "Back 9":
        list_of_holes = list(range(10, 19))
    elif holes == "18":
        list_of_holes = list(range(1, 19))
    else:
        pass
    return list_of_holes

def make_all_course_feature_lists(course_id, holes="18"):
    """
    Function pipelines all three calls to the three course feature tables
    :param course_id: id of course in course table
    :return: three lists made of distance, par, si
    """
    list_of_holes = make_hole_number_list(holes)
    course_distance_list = make_course_feature_using_course_id(course_id, DISTANCE_TABLE, holes)
    course_par_list = make_course_feature_using_course_id(course_id, PAR_TABLE, holes)
    course_si_list = make_course_feature_using_course_id(course_id, SI_TABLE, holes)
    return list_of_holes, course_distance_list, course_par_list, course_si_list

def make_round_score_card_csv(list_of_holes, course_distance_list, course_si_list, course_par_list):
    """
    Function will generate a .csv containing the course distance/si/par along with blank entries for strokes/putts/fir/gir
    :param course_distance_list: the distances of each hole on the course
    :param course_si_list: the stroke index of each hole on the course
    :param course_par_list: the par of each hole on the course
    :return: to_csv object that can be downloaded
    """
    round_score_card_template_df = pd.DataFrame()
    round_score_card_template_df["Hole"] = list_of_holes
    round_score_card_template_df["Distance"] = course_distance_list
    round_score_card_template_df["Stroke Index"] = course_si_list
    round_score_card_template_df["Par"] = course_par_list
    round_score_card_template_df["Shots"] = ""
    round_score_card_template_df["Putts"] = ""
    round_score_card_template_df["FIR"] = ""
    round_score_card_template_df["GIR"] = ""
    round_score_card_template_df_csv = round_score_card_template_df.to_csv(index=False)
    return round_score_card_template_df, round_score_card_template_df_csv

def make_pipeline_round_score_card_csv(course_name, holes="18"):
    """
    Function pipelines the process required to generate the round score card which includes course features such distance/si/par along with blank entries for strokes/putts/fir/gir
    :param course_name: course name
    :return: round score card csv
    """
    course_id = get_id_from_course_name(course_name)
    list_of_holes, course_distance_list, course_par_list, course_si_list = make_all_course_feature_lists(course_id, holes)
    round_score_card_template_df, round_score_card_template_df_csv = make_round_score_card_csv(list_of_holes, course_distance_list, course_si_list, course_par_list)
    return round_score_card_template_df, round_score_card_template_df_csv

### Getting Course Names

In [54]:
def make_alphabetical_course_name_list():
    """
    Function returns the row for a course feature table based on the id passed
    :param course_id: id of course in course table
    :return: sorted list of courses in course table
    """
    insert_command = """SELECT name FROM course;"""
    cursor.execute(insert_command,)
    returned_value = cursor.fetchall()
    feature_list = list(returned_value)
    sorted_feature_list = sorted([i[0] for i in feature_list])
    return sorted_feature_list

In [57]:
course_name_list = make_alphabetical_course_name_list()
course_name_list

['Aravell Golf Club',
 'Golf Costa Daurada',
 'Golf Round Test',
 'Loch Palm Golf Club',
 'Red Mountain Golf Club']

### Insert Round in Round Table

In [114]:
from datetime import datetime

# Insert course data into course table
def insert_round_in_round_table(course_id, user_id, date_played, tee_time, temperature, humidity, wind_speed, precipitation, weather_condition, holes_played):
    """
    Function inserts table information into the table table
    :param course_id: id of course
    :param user_id: id of user
    :param date_played: date played
    :param tee_time: time tee'd off
    :param temperature: average round temperature
    :param humidity: average round humidity
    :param wind_speed: average round wind speed
    :param precipitation: total round rainfall
    :param weather_condition: round weather condition
    :param holes_played: holes played that round
    :return: None
    """
    insert_command = """INSERT INTO round
                  (course_id, user_id, date_played, tee_time, temperature, humidity, wind_speed, precipitation, weather_condition, holes_played, date_created)
                  VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s);"""
    date_created = datetime.today().date()
    data_tuple = (course_id, user_id, date_played, tee_time, temperature, humidity, wind_speed, precipitation, weather_condition, holes_played, date_created)
    cursor_execute_tuple(insert_command, data_tuple)
    return None

In [116]:
course_id = 80
user_id = 12
date_played = datetime.today().date()
tee_time = 5
temperature = 4
humidity = 73
wind_speed = 39
precipitation = 80
weather_condition = "cold"
holes_played = 18


insert_round_in_round_table(course_id, user_id, date_played, tee_time, temperature, humidity, wind_speed, precipitation, weather_condition, holes_played)

TypeError: insert_round_in_round_table() takes 9 positional arguments but 10 were given